In [1]:
import sys
import scanpy as sc
import pandas as pd
import numpy as np
import os, glob
import pickle
import loompy

from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2

from pyscenic.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.utils import modules_from_adjacencies
from pyscenic.prune import prune2df, df2regulons, _distributed_calc
from pyscenic.aucell import aucell
from dask.diagnostics import ProgressBar
from distributed import LocalCluster, Client
import logging

In [2]:
sys.executable #double checking correct kernel is loaded

'/home/bob/anaconda3/envs/scenic0103/bin/python3'

In [3]:
#Use kernel installed with scenic_phase_1.yml for Phase 1

In [6]:
RESOURCES_FOLDER="/home/bob/SCENIC/pySCENIC/resources/" #location to store motifs and transcription factor tables/lists. get from pyscenic github repo
MOTIF_ANNOTATIONS_FNAME = os.path.join(RESOURCES_FOLDER, "motifs-v9-nr.hgnc-m0.001-o0.0.tbl")
HS_TFS_FNAME = os.path.join(RESOURCES_FOLDER, 'hs_hgnc_tfs.txt')

DATABASE_FOLDER = "/home/bob/SCENIC/pySCENIC/databases/" #location to store feather databases, from https://resources.aertslab.org/cistarget/
DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "hg19-*.mc9nr.feather") #only three from the 10 species comparison are needed, from hg19, mc9nr for humans

DATA_FOLDER="." #output directory
ADJACENCIES_FNAME = os.path.join(DATA_FOLDER, "10_species_adj.csv") #can change these output filenames, will output in DATA_FOLDER
MODULES_FNAME = os.path.join(DATA_FOLDER, "10_species_modules.p") #can change these output filenames, will output in DATA_FOLDER
MOTIFS_FNAME = os.path.join(DATA_FOLDER, "10_species_motifs.csv") #can change these output filenames, will output in DATA_FOLDER
REGULONS_FNAME = os.path.join(DATA_FOLDER, "10_species_regulons.p") #can change these output filenames, will output in DATA_FOLDER
AUC_FNAME = os.path.join(DATA_FOLDER, "10_species_aucell.csv") #can change these output filenames, will output in DATA_FOLDER

In [7]:
#load databases and tf info into environment

In [8]:
tf_names = load_tf_names(HS_TFS_FNAME)
db_fnames = glob.glob(DATABASES_GLOB)
def name(fname):
    return os.path.splitext(os.path.basename(fname))[0]
dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]
dbs #shows the 3 databases loaded

[FeatherRankingDatabase(name="hg19-tss-centered-5kb-10species.mc9nr"),
 FeatherRankingDatabase(name="hg19-500bp-upstream-10species.mc9nr"),
 FeatherRankingDatabase(name="hg19-tss-centered-10kb-10species.mc9nr")]

In [4]:
# set up dask cluster, remember to set proper port forwarding during ssh connection to forward the dashboard address to a local address

In [11]:
local_cluster = LocalCluster(n_workers=2,threads_per_worker=32,dashboard_address=':2345',memory_limit='64GB')
custom_client_coexp =  Client(local_cluster)
#each n_worker added multiplies the memory_limit, but each thread per worker does not affect ram usage. 
#we have a total of 64 threads over 32 cores
#10k cells needs 16GB ram per process
#20k cells needs 32GB ram per process
#60k cells needs 96GB ram per process

In [ ]:
# read data in as data frame, set type to a 16 bit int to save memory, signed int16s have a max value of 32,767

In [10]:
data = sc.read_h5ad("2020_2021_Cohort_Polyp_Cancer_Abnormals_Epi.h5ad").to_df().astype('int16') 

In [ ]:
#run grnboost2 using the dask cluster, takes a long time, but you can check the progress using the dask dashboard address

In [ ]:
%%time
adj = grnboost2(data, tf_names=tf_names, verbose=True,client_or_address=custom_client_coexp)

preparing dask client
parsing input
creating dask graph


/home/bob/anaconda3/envs/scenic0103/lib/python3.8/site-packages/arboreto/algo.py:214: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  expression_matrix = expression_data.as_matrix()


In [ ]:
adj.to_csv(ADJACENCIES_FNAME, index=False, sep='\t') #save results

In [ ]:
local_cluster.close()
custom_client_coexp.close() #close cluster

In [ ]:
#this part should be quick, it's well parallelized and generates the module file needed for the next steps. i usually mask the dropouts

In [12]:
%%time
modules = list(modules_from_adjacencies(adj, data, rho_mask_dropouts=True))
#bunch of nanny warnings show up if running in same notebook as coexpression


2021-04-13 10:45:40,180 - pyscenic.utils - INFO - Calculating Pearson correlations.

2021-04-13 10:45:41,819 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [True].

2021-04-13 10:49:22,108 - pyscenic.utils - INFO - Creating modules.


CPU times: user 1h 55min 9s, sys: 1h 42min 39s, total: 3h 37min 49s
Wall time: 5min 44s


In [13]:
len(modules) #dropout masking

8766

In [14]:
with open(MODULES_FNAME, "wb") as f: #save as pickle
    pickle.dump(modules, f)

In [57]:
####################################################################################################################################################################################

In [14]:
#PHASE 1 COMPLETE
#make sure you shut down your kernel here and switch into the scenic phase 2+3 conda environment and its respective kernel

In [59]:
####################################################################################################################################################################################

In [9]:
with open(MODULES_FNAME, 'rb') as f: #read in modules file
    modules = pickle.load(f)

In [10]:
len(modules) #check length and make sure it matches

8766

In [11]:
tf_names = load_tf_names(HS_TFS_FNAME)#load these files in again 
db_fnames = glob.glob(DATABASES_GLOB)
def name(fname):
    return os.path.splitext(os.path.basename(fname))[0]
dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]
dbs #shows the 3 databases loaded

[FeatherRankingDatabase(name="hg19-tss-centered-5kb-10species.mc9nr"),
 FeatherRankingDatabase(name="hg19-500bp-upstream-10species.mc9nr"),
 FeatherRankingDatabase(name="hg19-tss-centered-10kb-10species.mc9nr")]

In [ ]:
#next step is parallelized through multiprocess and not dask, it's also worse at memory management
#the ram used scales directly with the number of workers used, meaning that it is faster with more workers but if it hits the memory limit it will crash
#so always allocate more memory by using fewer workers to be safe

In [12]:
%%time
LOGGER = logging.getLogger('pyscenic')
LOGGER.setLevel(logging.CRITICAL)
with ProgressBar(): 
    df = prune2df(dbs, modules, MOTIF_ANNOTATIONS_FNAME,module_chunksize=50,num_workers=2)
#60K cells need 1 process
#30K cells need 2 processes

[########################################] | 100% Completed | 56min 57.8s
CPU times: user 2min 5s, sys: 24.1 s, total: 2min 29s
Wall time: 57min


In [13]:
# Save the enriched motifs and the discovered regulons to disk.
df.to_csv(MOTIFS_FNAME)

In [14]:
#convert to regulons
regulons = df2regulons(df)

Create regulons from a dataframe of enriched features.
Additional columns saved: []


In [15]:
len(regulons) #check length

174

In [ ]:
#save pickle file of regulons

In [16]:
with open(REGULONS_FNAME, "wb") as f:
    pickle.dump(regulons, f)

In [29]:
####################################################################################################################################################################################

In [30]:
#Phase 3 comes up next, you can use the same pyscenic 2 + 3 environment

In [20]:
####################################################################################################################################################################################

In [2]:
with open(REGULONS_FNAME, 'rb') as f:
    regulon = pickle.load(f)

In [3]:
len(regulon) #make sure it's the same length as before and it's the right file

154

In [21]:
data = sc.read_h5ad("2020_2021_Cohort_Polyp_Cancer_Abnormals_Epi.h5ad").to_df().astype('int16') #read in data of interest as dataframe again

In [ ]:
#run aucell, may also take up a lot of ram but is generally fast, may have to filter the input matrix in some cases if too large

In [23]:
auc_mtx = aucell(data, regulons, num_workers=32)

In [25]:
# Save the enriched motifs and the discovered regulons to disk.
auc_mtx.to_csv(AUC_FNAME)